In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Dropout,BatchNormalization,Embedding,TimeDistributed,Activation
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau





2025-12-20 10:45:03.006525: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766227503.183139      38 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766227503.234398      38 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
sentences3 = []

with open("/kaggle/input/subtitles-dataset/en (1).txt", "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= 500000:   
            break
        line = line.strip()
        if line:
            sentences3.append(line.lower())


train_sentences, val_sentences = train_test_split(sentences3, test_size=0.2, random_state=42)
print("Train sentences:", len(train_sentences))
print("Validation sentences:", len(val_sentences))

Train sentences: 400000
Validation sentences: 100000


In [3]:
tokenizer_ng3 = Tokenizer(oov_token="<OOV>")
tokenizer_ng3.fit_on_texts(train_sentences)
vocab_size_ng3 = len(tokenizer_ng3.word_index) + 1
print("Total vocab size:", vocab_size_ng3)


def build_ngram_sequences_unique(lines, tokenizer, max_len=None):
    seq_list = []
    for line in lines:
        t_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(t_list)):
            seq_list.append(t_list[:i+1])

    if not seq_list:
        return None, None, max_len

    if max_len is None:
        max_len = max(len(s) for s in seq_list)

    seq_list = pad_sequences(seq_list, maxlen=max_len, padding="pre")
    X_u, y_u = seq_list[:, :-1], seq_list[:, -1]
    return X_u, y_u, max_len


# Train set
x0_ng, y0_ng, maxlen_ng3 = build_ngram_sequences_unique(train_sentences, tokenizer_ng3)
print("Train samples:", x0_ng.shape, y0_ng.shape)
print("Max sequence length:", maxlen_ng3)

# Validation set
xv_ng, yv_ng, _ = build_ngram_sequences_unique(val_sentences, tokenizer_ng3, max_len=maxlen_ng3)

print("Validation samples:", xv_ng.shape, yv_ng.shape)


Total vocab size: 31572
Train samples: (1773893, 59) (1773893,)
Max sequence length: 60
Validation samples: (444740, 59) (444740,)


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, BatchNormalization, Dropout
from tensorflow.keras import regularizers
from keras.optimizers import Adam

model = Sequential()
model.add(Embedding(vocab_size_ng3, 512, input_length=maxlen_ng3-1))
model.add(GRU(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.1))
model.add(GRU(512, return_sequences=False, dropout=0.25))

model.add(Dense(512, activation="relu", kernel_regularizer=regularizers.l2(1e-4)))
model.add(Dropout(0.2))
model.add(Dense(256, activation="relu", kernel_regularizer=regularizers.l2(1e-4)))
model.add(Dropout(0.25))


model.add(Dense(vocab_size_ng3, activation="softmax"))

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=Adam(learning_rate=0.0003, clipnorm=1.0),
    metrics=["accuracy"]
)

In [11]:
model.build((None, maxlen_ng3-1))
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 59, 512)        │    16,164,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_4 (GRU)                     │ (None, 59, 512)        │     1,575,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_5 (GRU)                     │ (None, 512)            │     1,575,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 31572)          │     8,114,004 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,824,724 (106.14 MB)

 Trainable params: 27,824,724 (106.14 MB)

 Non-trainable params: 0 (0.00 B)

In [1]:

early_stop_ng = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
lr_scheduler_ng = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1)

history_ng = model.fit(
    x0_ng, y0_ng,
    epochs=30,
    batch_size=128,
    validation_data=(xv_ng, yv_ng),
    callbacks=[early_stop_ng, lr_scheduler_ng]
)

NameError: name 'EarlyStopping' is not defined

In [ ]:
from tensorflow.keras.models import load_model

# path to your model file (use quotes and correct directory)
model_path = "/kaggle/input/gru-model/tensorflow2/default/1/gru_mode.h5"

# load the model
model1 = load_model(model_path)

# verify load
model1.summary()


In [ ]:
final_loss, final_acc = model.evaluate(xv_ng, yv_ng, batch_size=128, verbose=1)

print("Final Validation Loss:", final_loss)
print("Final Validation Accuracy:", final_acc)


In [31]:
import numpy as np
print("Perplexity:", np.exp(model.evaluate(xv_ng, yv_ng)[0]))


  112/13899 ━━━━━━━━━━━━━━━━━━━━ 8:56 39ms/step - accuracy: 0.2698 - loss: 4.3563

KeyboardInterrupt: 

In [50]:
model.save("predict_model.h5")

In [ ]:
import pickle
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer3, f)



In [ ]:


import pickle


with open("tokenizer.pkl", "rb") as f:
    tok3 = pickle.load(f)


In [ ]:
from tensorflow.keras.models import load_model

# path to your model file (use quotes and correct directory)
model_path = "/kaggle/input/gru-model/tensorflow2/default/1/gru_mode.h5"

# load the model
model1 = load_model(model_path)

# verify load
model1.summary()


In [32]:
def output(seed_text, model, max_len, tokenizer, next_words=20):
    in_text = seed_text

    for _ in range(next_words):
        seq = tokenizer.texts_to_sequences([in_text])[0]  # fixed typo
        seq = seq[-(max_len-1):]  # keep last max_len-1 tokens
        seq = np.pad(seq, (max_len-1-len(seq), 0), "constant")

        y_pred = model.predict(seq.reshape(1, max_len-1), verbose=0)  # fixed typo
        next_index = np.argmax(y_pred)
        next_word = tokenizer.index_word[next_index]

        in_text += " " + next_word

    return in_text


In [53]:
import numpy as np

def top_k_temperature_sampling(preds, k=40, temperature=0.8):
    preds = np.asarray(preds).astype("float64")

    # temperature scaling
    preds = np.log(preds + 1e-9) / temperature

    # keep only top-k probabilities
    top_k_indices = np.argpartition(preds, -k)[-k:]
    top_k_logits = preds[top_k_indices]

    # softmax
    exp_preds = np.exp(top_k_logits)
    probs = exp_preds / np.sum(exp_preds)

    # sample
    return np.random.choice(top_k_indices, p=probs)


def output(
    seed_text,
    model,
    max_len,
    tokenizer,
    next_words=20,
    temperature=0.8,
    top_k=40
):
    in_text = seed_text

    for _ in range(next_words):
        seq = tokenizer.texts_to_sequences([in_text])[0]
        seq = seq[-(max_len - 1):]
        seq = np.pad(seq, (max_len - 1 - len(seq), 0), "constant")

        preds = model.predict(seq.reshape(1, max_len - 1), verbose=0)[0]

        next_index = top_k_temperature_sampling(
            preds,
            k=top_k,
            temperature=temperature
        )

        next_word = tokenizer.index_word.get(next_index, "")
        in_text += " " + next_word

    return in_text


In [82]:
output(
    seed_text="i love",
    model=model,
    max_len=maxlen_ng3,
    tokenizer=tokenizer_ng3,
    next_words=5,
    temperature=0.8,
    top_k=10
)


'i love you all right now i'

In [52]:
print(output(
    seed_text="i love",
    next_words=60,
    model=model,
    max_len=maxlen_ng3,
    tokenizer=tokenizer_ng3,
   
))

i love you too much to you and i don't know what i want to do to you the way you know that i was a little bit of a little bit of a little bit of the same and i was a little bit of a little bit of a little bit of a little bit of the same to be


In [ ]:
import numpy as np
print("Perplexity:", np.exp(model1.evaluate(X_val1, y_val1)[0]))


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, BatchNormalization, Dropout
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# -----------------------------------------------------
# LOAD DATA
# -----------------------------------------------------
sentences_small = []

with open("/kaggle/input/subtitles-dataset/en (1).txt", "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= 100000:
            break
        line = line.strip()
        if line:
            sentences_small.append(line.lower())

train_sentences5, val_sentences5 = train_test_split(sentences_small, test_size=0.2, random_state=42)
print("Train sentences:", len(train_sentences5))
print("Validation sentences:", len(val_sentences5))

# -----------------------------------------------------
# TOKENIZER (new tokenizer for this model)
# -----------------------------------------------------
tokenizer5 = Tokenizer(oov_token="<OOV>")
tokenizer5.fit_on_texts(train_sentences5)

total_words_5 = len(tokenizer5.word_index) + 1
print("Total vocab size:", total_words_5)

# -----------------------------------------------------
# N-GRAM SEQUENCE BUILDER
# -----------------------------------------------------
def build_ngram_sequences(lines, tokenizer, max_len=None):
    sequences = []
    for line in lines:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram = token_list[:i+1]
            sequences.append(n_gram)

    if not sequences:
        return None, None, max_len

    # Calculate max_len automatically OR use given
    if max_len is None:
        max_len = max(len(x) for x in sequences)

    sequences = pad_sequences(sequences, maxlen=max_len, padding="pre")
    X, y = sequences[:, :-1], sequences[:, -1]
    return X, y, max_len

# -----------------------------------------------------
# BUILD TRAIN SEQUENCES
# -----------------------------------------------------
X5, y5, max_len_seq5 = build_ngram_sequences(train_sentences5, tokenizer5)
print("Train samples:", X5.shape, y5.shape)
print("Max sequence length:", max_len_seq5)

# -----------------------------------------------------
# VALIDATION SEQUENCES (use same max_len)
# -----------------------------------------------------
X_val5, y_val5, _ = build_ngram_sequences(val_sentences5, tokenizer5, max_len=max_len_seq5)
print("Validation samples:", X_val5.shape, y_val5.shape)

# -----------------------------------------------------
# MODEL
# -----------------------------------------------------
model = Sequential()
model.add(Embedding(total_words_5, 128, input_length=max_len_seq5-1))
model.add(GRU(128, return_sequences=True, dropout=0.1))
model.add(GRU(128, dropout=0.3))

model.add(Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Dense(total_words_5, activation="softmax"))

from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=0.001, clipnorm=1.0)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=opt,
    metrics=["accuracy"]
)


model.summary()




In [ ]:
import numpy as np

print("Any empty rows in X5?  →", np.any(np.all(X5 == 0, axis=1)))


In [ ]:
print("Any invalid y labels? →", np.any((y5 == 0) | (y5 >= total_words_5)))
print("Min y:", y5.min(), "Max y:", y5.max(), "Vocab:", total_words_5)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)

lr_scheduler = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=2,
    verbose=1
)

history = model.fit(
    X5, y5,
    epochs=30,
    batch_size=64,
    validation_data=(X_val5, y_val5),
    callbacks=[early_stop, lr_scheduler]
)


In [ ]:
import numpy as np
print("Perplexity:", np.exp(model.evaluate(X_val5, y_val5)[0]))
